An implimentation of the pseudo code found in:
A Revealing Introduction to Hidden Markov Models
Mark Stamp
Department of Computer Science
San Jose State University
January 12, 2018
https://www.cs.sjsu.edu/~stamp/RUA/HMM.pdf
    
In this example, A is the state transition matrix, B the observation probability matrix, O the observation matrix, and \pi the intial state matrix.

In [7]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Feb  2 13:34:35 2018

@author: rbnsnsd2
"""
import numpy as np


"""
Given the observations
"""
O = np.array([[0,1,0,2]]) #each observation state in sequence
T = len(O[0]) #number of observations in sequence
    
"""
Initialize the matrices
"""
A = np.array([[.7,.3],
              [.4,.6]]) #Rows are present state
B = np.array([[.1,.4,.5],
              [.7,.2,.1]]) #rows are present state
pi = np.array([.6,.4]) #likelihood of intial state

"""
Global limits
"""
N = np.shape(B)[0]
M = np.shape(B)[1]


def init_matrices(O, N):
    """
    Create random inital values for the matrices
    that are row stocastic
    O is the observed state sequence
    N is the number of available hidden states
    """
    T = len(O[0])
    M = len(np.unique(O))
    A = np.random.rand(N,N)
    A = A/A.sum(axis=1)[:,None]
    B = np.random.rand(N,M)
    B = B/B.sum(axis=1)[:,None]
    pi = np.random.rand(N)
    pi = pi/pi.sum()
    c = np.zeros((T))
    alpha = np.zeros((T,N))
    beta = np.zeros((T,N))
    gamma = np.zeros((T,N))
    digam = np.zeros((T,N,N))
    return A, B, pi, alpha, beta, gamma, digam, M, T, c

#A, B, pi, M = init_matrices(O,N)

"""
Intialize the iterators
"""
maxIters = 100
iters = 0
oldLogProb = -10**100 #Minus infinity...

c = np.zeros((T))
alpha = np.zeros((T,N))
beta = np.zeros((T,N))
gamma = np.zeros((T,N))
digam = np.zeros((T,N,N))


"""
Alpha pass
"""
def apass(A,B,pi,alpha,N,T,c):
    """Computer alpha[0,i]"""
    c[0] = 0
    for i in range(N):
        alpha[0,i] = pi[i]*B[i,O[0,0]]
        c[0] = c[0] + alpha[0,i]
        
    """Scale alpha[0,i]"""
    c[0] = 1/c[0]
    for i in range(N):
        alpha[0,i] = c[0]*alpha[0,i]
    
    """Compute alpha[t,i]"""
    for t in range(1,T): 
        c[t] = 0
        
        for i in range(N): 
            alpha[t,i] = 0
            for j in range(N): 
                alpha[t,i] = alpha[t,i] + alpha[t-1,j]*A[j,i]
            alpha[t,i] = alpha[t,i]*B[i,O[0,t]]
            c[t] = c[t] + alpha[t,i]
            
        c[t] = 1/c[t] #Scale alpha[t,i]
        for i in range(N):
            alpha[t,i] = c[t]*alpha[t,i]
    return alpha, c


"""
Beta pass
"""
def bpass(A,B,pi,beta,N,T,c):
    """Set beta[t,i]=1 scaled by c[t-1]"""
    for i in range(N):
        beta[T-1,i] = c[T-1]
        
    """Beta pass"""
    for t in range(T-2,-1,-1):
        for i in range(N):
            beta[t,i] = 0
            for j in range(N):
                beta[t,i] = beta[t,i] + A[i,j]*B[j,O[0,t+1]]*beta[t+1,j]
            #Scale beta[t,i] with the same as alpha[t,i]
            beta[t,i] = c[t]*beta[t,i]
            
    return beta, c


"""
Digamma
"""
def digamma(A,B,pi,alpha,beta,gamma,digam,N,T):
    """No normalization since alpha beta already scaled"""
    for t in range(T-1):
        for i in range(N):
            gamma[t,i] = 0
            for j in range(N):
                digam[t,i,j] = alpha[t,i]*A[i,j]*B[j,O[0,t+1]]*beta[t+1,j]
                gamma[t,i] = gamma[t,i] + digam[t,i,j]
    """Special case for gamma[t-1,i]"""
    for i in range(N):
        gamma[T-1,i] = alpha[T-1,i]
    
    return gamma, digam

"""
Re-estimate A, B, pi
"""
def re_est(A,B,pi,gamma,digam):
    """re-estimate pi"""
    for i in range(N):
        pi[i] = gamma[0,i]
        
    """Re-estimate A"""
    for i in range(N):
        for j in range(N):
            numer = 0
            denom = 0
            for t in range(T-1):
                numer = numer + digam[t,i,j]
                denom = denom + gamma[t,i]
            A[i,j] = numer/denom
            
    """re-estimate B"""
    for i in range(N):
        for j in range(M):
            numer = 0
            denom = 0
            for t in range(T):
                if O[0,t] == j: numer = numer + gamma[t,i]
                denom = denom + gamma[t,i]
            B[i,j] = numer/denom
    return A, B, pi

"""
Compute log P(O|lambda)
"""
def logprob(c):
    return -np.sum(np.log(c))      
         
"""
Put it all together
"""
def markov(O,N):
    iters = 0
    logProb = 0
    delta = 1
    A, B, pi, alpha, beta, gamma, digam, M, T, c = init_matrices(O,N)
    while iters <= maxIters and logProb >= oldLogProb and delta >= 0.000001:
        alpha, c = apass(A,B,pi,alpha,N,T,c)
        beta, c = bpass(A,B,pi,beta,N,T,c)
        gamma, digam = digamma(A,B,pi,alpha,beta,gamma,digam,N,T)
        A, B, pi = re_est(A,B,pi,gamma,digam)
        delta1 = logProb
        logProb = logprob(c)
        delta = np.absolute(delta1 - logProb)
        iters = iters + 1
    print("Interations: ", iters)
    return A,B,pi,alpha,beta,gamma,digam
    
            
"""
Find P(O|lambda)
"""
def p_obs_lambda(alpha):
    return np.sum(alpha[T-1,:])

"""
Find most likely hidden states
"""
def p_state(gamma):
    return np.argmax(gamma,axis=1)
 

The converged model after X iterations, from the observations (O) and assumed available states (N), is then:

In [8]:
A, B, pi, alpha, beta, gamma, digam = markov(O,N)
print("A = {}\nB = {}".format(A,B))

Interations:  10
A = [[  9.55934328e-37   1.00000000e+00]
 [  1.00000000e+00   2.88348778e-70]]
B = [[  1.00000000e+00   1.09271581e-73   9.55934328e-37]
 [  1.44174388e-70   5.00000000e-01   5.00000000e-01]]
